## 1.Python libraries

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, avg, count

### Initialize Spark and Cassandra session

In [ ]:
spark = SparkSession.builder\
    .appName("MovieLensAnalysis")\
    .enableHiveSupport()\
    .config("spark.sql.shuffle.partitions", 8)\
    .getOrCreate()

print("Spark version:", spark.version)

('Spark version:', u'2.3.0.2.6.5.0-292')


In [ ]:
df = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="users", keyspace="movielens") \
    .load()
df.show(5)

+-------+---+------+----------+-----+
|user_id|age|gender|occupation|  zip|
+-------+---+------+----------+-----+
|    496| 21|     F|   student|55414|
|    250| 29|     M| executive|95110|
|    675| 34|     M|     other|28814|
|    175| 26|     F| scientist|21911|
|    637| 30|     M|     other|74101|
+-------+---+------+----------+-----+
only showing top 5 rows



## 2.Parse the u.user file into HDFS

In [ ]:
hdfs dfs -mkdir -p /user/maria_dev/ml-100k/

In [ ]:
hdfs dfs -put -f /tmp/ml-100k/u.user /user/maria_dev/ml-100k/u.user
hdfs dfs -put -f /tmp/ml-100k/u.item /user/maria_dev/ml-100k/u.item
hdfs dfs -put -f /tmp/ml-100k/u.data /user/maria_dev/ml-100k/u.data
hdfs dfs -ls /user/maria_dev/ml-100k/

Found 7 items
drwxr-xr-x   - zeppelin hdfs          0 2025-06-13 13:02 /user/maria_dev/ml-100k/
-rw-r--r--   1 zeppelin hdfs    1979173 2025-06-18 07:47 /user/maria_dev/ml-100k/u.data
-rw-r--r--   1 zeppelin hdfs    1979173 2025-06-13 13:14 /user/maria_dev/ml-100k/u.data
-rw-r--r--   1 zeppelin hdfs     236344 2025-06-18 07:47 /user/maria_dev/ml-100k/u.item
-rw-r--r--   1 zeppelin hdfs     236344 2025-06-13 13:14 /user/maria_dev/ml-100k/u.item
-rw-r--r--   1 zeppelin hdfs      22628 2025-06-18 07:47 /user/maria_dev/ml-100k/u.user
-rw-r--r--   1 zeppelin hdfs      22628 2025-06-13 13:14 /user/maria_dev/ml-100k/u.user


## 3. Load, Read, and Create Resilient Distributed Dataset (RDD) Objects.

### Loading from HDFS as RDD

In [ ]:
user_rdd = sc.textFile("hdfs:///user/maria_dev/ml-100k/u.user")
user_rdd.take(5)

[u'1|24|M|technician|85711', u'2|53|F|other|94043', u'3|23|M|writer|32067', u'4|24|M|technician|43537', u'5|33|F|other|15213']


In [ ]:
item_rdd = sc.textFile("hdfs:///user/maria_dev/ml-100k/u.item")
item_rdd.take(5)

[u'1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0', u'2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0', u'3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0', u'4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0', u'5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0']


In [ ]:
data_rdd = sc.textFile("hdfs:///user/maria_dev/ml-100k/u.data")
data_rdd.take(5)

[u'196\t242\t3\t881250949', u'186\t302\t3\t891717742', u'22\t377\t1\t878887116', u'244\t51\t2\t880606923', u'166\t346\t1\t886397596']


### Parsing RDD

In [ ]:
user_parsed_rdd = user_rdd.map(lambda line: line.split('|'))
user_parsed_rdd.take(5)

[[u'1', u'24', u'M', u'technician', u'85711'], [u'2', u'53', u'F', u'other', u'94043'], [u'3', u'23', u'M', u'writer', u'32067'], [u'4', u'24', u'M', u'technician', u'43537'], [u'5', u'33', u'F', u'other', u'15213']]


In [ ]:
item_parsed_rdd = item_rdd.map(lambda line: line.split('|'))
item_parsed_rdd.take(3)

[[u'1', u'Toy Story (1995)', u'01-Jan-1995', u'', u'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)', u'0', u'0', u'0', u'1', u'1', u'1', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0'], [u'2', u'GoldenEye (1995)', u'01-Jan-1995', u'', u'http://us.imdb.com/M/title-exact?GoldenEye%20(1995)', u'0', u'1', u'1', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'1', u'0', u'0'], [u'3', u'Four Rooms (1995)', u'01-Jan-1995', u'', u'http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'1', u'0', u'0']]


In [ ]:
data_parsed_rdd = data_rdd.map(lambda line: line.split('\t'))
data_parsed_rdd.take(5)

[[u'196', u'242', u'3', u'881250949'], [u'186', u'302', u'3', u'891717742'], [u'22', u'377', u'1', u'878887116'], [u'244', u'51', u'2', u'880606923'], [u'166', u'346', u'1', u'886397596']]


### Convert to a structured dictionary

In [ ]:
user_dict_rdd = user_parsed_rdd.map(lambda arr: {
    "user_id": int(arr[0]),
    "age": int(arr[1]),
    "gender": arr[2],
    "occupation": arr[3],
    "zip_code": arr[4]
})

user_dict_rdd.take(5)

[{'gender': u'M', 'age': 24, 'occupation': u'technician', 'user_id': 1, 'zip_code': u'85711'}, {'gender': u'F', 'age': 53, 'occupation': u'other', 'user_id': 2, 'zip_code': u'94043'}, {'gender': u'M', 'age': 23, 'occupation': u'writer', 'user_id': 3, 'zip_code': u'32067'}, {'gender': u'M', 'age': 24, 'occupation': u'technician', 'user_id': 4, 'zip_code': u'43537'}, {'gender': u'F', 'age': 33, 'occupation': u'other', 'user_id': 5, 'zip_code': u'15213'}]


In [ ]:
genres = [
    "unknown", "Action", "Adventure", "Animation", "Children", "Comedy", "Crime", "Documentary", "Drama",
    "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]

item_dict_rdd = item_parsed_rdd.map(lambda arr: {
    "movie_id": int(arr[0]),
    "movie_title": arr[1],
    "release_date": arr[2],
    "video_release_date": arr[3],
    "IMDb_URL": arr[4],
    "genres": [genres[i] for i in range(19) if int(arr[5+i]) == 1]
})

item_dict_rdd.take(1)

[{'video_release_date': u'', 'genres': ['Animation', 'Children', 'Comedy'], 'movie_id': 1, 'release_date': u'01-Jan-1995', 'movie_title': u'Toy Story (1995)', 'IMDb_URL': u'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)'}]


In [ ]:
data_dict_rdd = data_parsed_rdd.map(lambda arr: {
    "user_id": int(arr[0]),
    "movie_id": int(arr[1]),
    "rating": int(arr[2]),
    "timestamp": int(arr[3])
})
data_dict_rdd.take(3)

[{'rating': 3, 'user_id': 196, 'movie_id': 242, 'timestamp': 881250949}, {'rating': 3, 'user_id': 186, 'movie_id': 302, 'timestamp': 891717742}, {'rating': 1, 'user_id': 22, 'movie_id': 377, 'timestamp': 878887116}]


## 4. Convert the RDD objects into DataFrames.

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

user_schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("occupation", StringType(), True),
    StructField("zip_code", StringType(), True)
])

users_df = spark.createDataFrame(user_dict_rdd, schema=user_schema)
users_df.show(5)

+-------+---+------+----------+--------+
|user_id|age|gender|occupation|zip_code|
+-------+---+------+----------+--------+
|      1| 24|     M|technician|   85711|
|      2| 53|     F|     other|   94043|
|      3| 23|     M|    writer|   32067|
|      4| 24|     M|technician|   43537|
|      5| 33|     F|     other|   15213|
+-------+---+------+----------+--------+
only showing top 5 rows



In [ ]:
from pyspark.sql.types import ArrayType, StringType, StructType, StructField, IntegerType

item_schema = StructType([
    StructField("movie_id", IntegerType(), True),
    StructField("movie_title", StringType(), True),
    StructField("release_date", StringType(), True),
    StructField("video_release_date", StringType(), True),
    StructField("IMDb_URL", StringType(), True),
    StructField("genres", ArrayType(StringType()), True)
])

items_df = spark.createDataFrame(item_dict_rdd, schema=item_schema)
items_df.show(5, truncate=False)

+--------+-----------------+------------+------------------+------------------------------------------------------+-----------------------------+
|movie_id|movie_title      |release_date|video_release_date|IMDb_URL                                              |genres                       |
+--------+-----------------+------------+------------------+------------------------------------------------------+-----------------------------+
|1       |Toy Story (1995) |01-Jan-1995 |                  |http://us.imdb.com/M/title-exact?Toy%20Story%20(1995) |[Animation, Children, Comedy]|
|2       |GoldenEye (1995) |01-Jan-1995 |                  |http://us.imdb.com/M/title-exact?GoldenEye%20(1995)   |[Action, Adventure, Thriller]|
|3       |Four Rooms (1995)|01-Jan-1995 |                  |http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|[Thriller]                   |
|4       |Get Shorty (1995)|01-Jan-1995 |                  |http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|[Action, 

In [ ]:
data_schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("movie_id", IntegerType(), True),
    StructField("rating", IntegerType(), True),
    StructField("timestamp", IntegerType(), True)
])

ratings_df = spark.createDataFrame(data_dict_rdd, schema=data_schema)
ratings_df.show(5)

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
+-------+--------+------+---------+
only showing top 5 rows



In [ ]:
users_df.createOrReplaceTempView("users")
items_df.createOrReplaceTempView("items")
ratings_df.createOrReplaceTempView("ratings")

## 5. Write the DataFrame into the Keyspace database

In [ ]:
df_check = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="users", keyspace="movielens") \
    .load()
df_check.show(5)

+-------+---+------+----------+-----+
|user_id|age|gender|occupation|  zip|
+-------+---+------+----------+-----+
|    496| 21|     F|   student|55414|
|    250| 29|     M| executive|95110|
|    675| 34|     M|     other|28814|
|    175| 26|     F| scientist|21911|
|    637| 30|     M|     other|74101|
+-------+---+------+----------+-----+
only showing top 5 rows



In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS movielens")

DataFrame[]


In [ ]:
users_df.write.mode("overwrite").saveAsTable("movielens.users")
items_df.write.mode("overwrite").saveAsTable("movielens.items")
ratings_df.write.mode("overwrite").saveAsTable("movielens.ratings")

In [ ]:
spark.sql("SELECT * FROM movielens.users LIMIT 5").show()

+-------+---+------+----------+--------+
|user_id|age|gender|occupation|zip_code|
+-------+---+------+----------+--------+
|    472| 24|     M|   student|   87544|
|    473| 29|     M|   student|   94708|
|    474| 51|     M| executive|   93711|
|    475| 30|     M|programmer|   75230|
|    476| 28|     M|   student|   60440|
+-------+---+------+----------+--------+



## 6. Read the table back into a new DataFrame.

In [ ]:
# Read users table from Cassandra
users_new_df = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="users", keyspace="movielens") \
    .load()
users_new_df.show(5)


+-------+---+------+----------+-----+
|user_id|age|gender|occupation|  zip|
+-------+---+------+----------+-----+
|    496| 21|     F|   student|55414|
|    250| 29|     M| executive|95110|
|    675| 34|     M|     other|28814|
|    175| 26|     F| scientist|21911|
|    637| 30|     M|     other|74101|
+-------+---+------+----------+-----+
only showing top 5 rows



In [ ]:
users_new_df = spark.read.table("movielens.users")
users_new_df.show(5)

+-------+---+------+----------+--------+
|user_id|age|gender|occupation|zip_code|
+-------+---+------+----------+--------+
|    472| 24|     M|   student|   87544|
|    473| 29|     M|   student|   94708|
|    474| 51|     M| executive|   93711|
|    475| 30|     M|programmer|   75230|
|    476| 28|     M|   student|   60440|
+-------+---+------+----------+--------+
only showing top 5 rows



In [ ]:
items_new_df = spark.read.table("movielens.items")
items_new_df.show(5, truncate=False)

+--------+------------------------+------------+------------------+-----------------------------------------------------------------+-------------------------+
|movie_id|movie_title             |release_date|video_release_date|IMDb_URL                                                         |genres                   |
+--------+------------------------+------------+------------------+-----------------------------------------------------------------+-------------------------+
|839     |Loch Ness (1995)        |01-Jan-1995 |                  |http://us.imdb.com/M/title-exact?Loch%20Ness%20(1995)            |[Horror, Thriller]       |
|840     |Last Man Standing (1996)|20-Sep-1996 |                  |http://us.imdb.com/M/title-exact?Last%20Man%20Standing%20(1996/I)|[Action, Drama, Western] |
|841     |Glimmer Man, The (1996) |04-Oct-1996 |                  |http://us.imdb.com/M/title-exact?Glimmer%20Man,%20The%20(1996)   |[Action, Thriller]       |
|842     |Pollyanna (1960)        |01-Ja

In [ ]:
ratings_new_df = spark.read.table("movielens.ratings")
ratings_new_df.show(5)

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
+-------+--------+------+---------+
only showing top 5 rows



## i) Calculate the Average Rating for Each Movie

The main goal of this step is to compute the average rating for every movie in the MovieLens 100K dataset.
By doing so, I can understand which movies are generally more popular or appreciated by users, and which ones might have received a less favorable response.
To achieve this, I use Spark SQL to join the `ratings` and `items` tables, group the data by `movie_id` and `movie_title`, and compute:
- The average user rating for each movie (`AVG(r.rating)`)
- The total number of ratings each movie received (`COUNT(*)`)

In [ ]:
avg_rating_with_name_df = spark.sql("""
    SELECT
        i.movie_id,
        i.movie_title,
        AVG(r.rating) AS avg_rating,
        COUNT(*) AS rating_count
    FROM ratings r
    JOIN items i ON r.movie_id = i.movie_id
    GROUP BY i.movie_id, i.movie_title
    ORDER BY i.movie_id
""")
avg_rating_with_name_df.show(10, truncate=False)

+--------+----------------------------------------------------+------------------+------------+
|movie_id|movie_title                                         |avg_rating        |rating_count|
+--------+----------------------------------------------------+------------------+------------+
|1       |Toy Story (1995)                                    |3.8783185840707963|452         |
|2       |GoldenEye (1995)                                    |3.2061068702290076|131         |
|3       |Four Rooms (1995)                                   |3.033333333333333 |90          |
|4       |Get Shorty (1995)                                   |3.550239234449761 |209         |
|5       |Copycat (1995)                                      |3.302325581395349 |86          |
|6       |Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|3.576923076923077 |26          |
|7       |Twelve Monkeys (1995)                               |3.798469387755102 |392         |
|8       |Babe (1995)                   

For each movie, I group the ratings by the movie ID and movie title. Then calculate the average rating and the total number of ratings for each movie.  
This allows me to see not only how well a movie is rated on average, but also how many users have contributed to its score.

After running the query, I display the results in a table. This table shows the movie ID, the movie title, the average rating, and the number of ratings for each movie.  

#### Output Result Analysis
The table above presents the average rating and the number of ratings for each of the first 10 movies in the dataset.  
Several insights can be drawn from these results:

- **Most movies have average ratings between 3 and 4**, indicating that user opinions are generally positive.
- For example, *Toy Story (1995)* has an average rating of 3.88 with 452 ratings, suggesting that it is both popular and well-liked.
- Movies like *Babe (1995)* and *Dead Man Walking (1995)* also receive high average scores (above 3.9) with a substantial number of ratings.
- On the other hand, movies such as *Shanghai Triad* have far fewer ratings (only 26), which means their average score may not be as reliable or representative.
- The **rating_count** column is important: a high average rating with very few votes may not indicate universal appeal, while movies with both a high rating and many ratings are more robustly popular.
- These statistics provide a good overview of the distribution of movie popularity and user preferences in the dataset, and they serve as a foundation for more advanced analysis such as genre trends or recommendation systems.


## ii) Identifying the Top Ten Movies with the Highest Average Ratings

In this step, aiming to find the top ten movies with the highest average ratings.  
I use SQL to aggregate the movie ratings data. First, join the `ratings` and `items` tables. The `ratings` table contains user ratings for each movie. The `items` table provides the movie titles and other metadata.

Group the data by `movie_id` and `movie_title`. For each group, calculating the average rating and the total number of ratings.  
The average rating shows how much users enjoyed a particular movie. The rating count tells me how many users contributed to this score.

After grouping and aggregation, sort the movies in descending order by their average rating. Using the `LIMIT 10` clause to select only the top ten movies.  
This approach highlights the highest-rated movies in the dataset.

It is important to note that I can add an extra filter on the rating count. For example, I might only consider movies that have received at least 50 ratings.  
This filter helps to avoid bias from movies with very few but very high ratings. For now, Display the top ten movies based only on their average scores.

The output table presents the movie ID, the movie title, the average rating, and the number of ratings for each of the top ten movies.  

In [ ]:
top10_movies_df = spark.sql("""
    SELECT
        i.movie_id,
        i.movie_title,
        AVG(r.rating) AS avg_rating,
        COUNT(*) AS rating_count
    FROM ratings r
    JOIN items i ON r.movie_id = i.movie_id
    GROUP BY i.movie_id, i.movie_title
    HAVING rating_count >= 50
    ORDER BY avg_rating DESC, rating_count DESC
    LIMIT 10
""")
top10_movies_df.show(truncate=False)

+--------+------------------------------------------------------+------------------+------------+
|movie_id|movie_title                                           |avg_rating        |rating_count|
+--------+------------------------------------------------------+------------------+------------+
|408     |Close Shave, A (1995)                                 |4.491071428571429 |112         |
|318     |Schindler's List (1993)                               |4.466442953020135 |298         |
|169     |Wrong Trousers, The (1993)                            |4.466101694915254 |118         |
|483     |Casablanca (1942)                                     |4.45679012345679  |243         |
|114     |Wallace & Gromit: The Best of Aardman Animation (1996)|4.447761194029851 |67          |
|64      |Shawshank Redemption, The (1994)                      |4.445229681978798 |283         |
|603     |Rear Window (1954)                                    |4.3875598086124405|209         |
|12      |Usual Susp

#### Output Result Analysis

The table above lists the top ten movies with the highest average ratings, considering only movies that have received at least 50 user ratings.  
This approach ensures that the results are reliable and not skewed by movies with just a handful of votes.

- All of the listed movies have average ratings above 4.3, which means they are highly appreciated by users.
- Classics such as *Schindler's List (1993)*, *Rear Window (1954)*, and *Casablanca (1942)* appear among the top performers, reflecting their strong reputation and lasting appeal.
- Animation and family-friendly titles like *Wallace & Gromit: The Best of Aardman Animation (1996)* and *Wrong Trousers, The (1993)* are also highly rated, suggesting that well-made animated films are well received by the user community.
- *Shawshank Redemption, The (1994)* and *Star Wars (1977)* are both widely known and maintain extremely high average ratings with a large number of ratings (over 280 and 580 respectively), which demonstrates robust and broad-based popularity.
- The **rating_count** column shows that all these movies have a substantial number of ratings, making the averages meaningful and reliable.

## iii) Find the users who have rated at least 50 movies and identify their favourite movie genres

This analysis focuses on users who have demonstrated a high level of activity by rating at least 50 movies.  
The results are shown in several steps:

- First, I identified the most active users by grouping ratings by user ID and counting the number of movies each user rated. Only users with at least 50 ratings were included in the next step.
- Then, I counted, for each active user, how many movies they rated in each of the 19 available genres. This produced a wide table, where each row corresponds to a user and each column (after user ID) gives the count of ratings for one genre.
- Finally, I determined each user's favourite genre by finding which genre they had rated the most movies in. If a user had a tie for most-rated genres, all such genres were shown.

### Find users with ≥50 ratings

In [ ]:
active_users_df = spark.sql("""
    SELECT
        user_id,
        COUNT(*) AS num_rated_movies
    FROM ratings
    GROUP BY user_id
    HAVING num_rated_movies >= 50
""")
active_users_df.createOrReplaceTempView("active_users")
active_users_df.show(10)

+-------+----------------+
|user_id|num_rated_movies|
+-------+----------------+
|    186|              92|
|    299|             280|
|     38|             121|
|    157|              51|
|     13|             636|
|    198|             181|
|     18|             277|
|    232|              93|
|    161|              58|
|    148|              65|
+-------+----------------+
only showing top 10 rows



### The number of movie ratings per genre by each active user

In [ ]:

genres = [
    "unknown", "Action", "Adventure", "Animation", "Children", "Comedy", "Crime",
    "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery",
    "Romance", "Sci-Fi", "Thriller", "War", "Western"
]

genre_sum_expr = ',\n    '.join([
    "SUM(CAST(array_contains(i.genres, '{0}') AS INT)) AS `{0}`".format(genre)
    for genre in genres
])

sql_query = """
SELECT
    r.user_id,
    {genre_sum}
FROM ratings r
JOIN items i ON r.movie_id = i.movie_id
JOIN active_users u ON r.user_id = u.user_id
GROUP BY r.user_id
""".format(genre_sum=genre_sum_expr)

user_genre_count_df = spark.sql(sql_query)
user_genre_count_df.createOrReplaceTempView("user_genre_count")
user_genre_count_df.show(10)

+-------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|user_id|unknown|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+-------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|     12|      0|    14|        7|        1|       3|    13|    2|          0|   24|      0|        0|     1|      3|      3|     13|     7|      12|  7|      2|
|     13|      0|   156|       73|       16|      52|   188|   45|         13|  218|     12|       14|    75|     31|     33|    105|    65|     121| 41|     13|
|     14|      0|    18|       14|        6|       6|    35|    6|          2|   39|      1|        3|     1|      5|      6|     21|    13|      13| 10|      1|
|     18|      0|    25|    

### Find out each user's favorite movie genre

In [ ]:
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql.functions import udf

In [ ]:
genres=[
    "unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]

In [ ]:
def fav_genre_and_count(*cols):
    # cols: 19 types of ratings
    counts = list(cols)
    max_count = max(counts)
    favs = [genres[i] for i, v in enumerate(counts) if v == max_count]
    return ",".join(favs)

fav_genre_udf = udf(fav_genre_and_count, StringType())

In [ ]:
# Pass the 19 fields into the udf one by one
user_genre_count_df = user_genre_count_df.withColumn(
    "fav_genre",
    fav_genre_udf(*[user_genre_count_df[g] for g in genres]))

In [ ]:
# output max_count
def max_count_only(*cols):
    return int(max(list(cols)))
max_count_udf = udf(max_count_only, IntegerType())
user_genre_count_df = user_genre_count_df.withColumn(
    "max_count",
    max_count_udf(*[user_genre_count_df[g] for g in genres]))

In [ ]:
# Display results
user_genre_count_df.select("user_id", "fav_genre", "max_count").show(10, truncate=False)

+-------+---------+---------+
|user_id|fav_genre|max_count|
+-------+---------+---------+
|12     |Drama    |24       |
|13     |Drama    |218      |
|14     |Drama    |39       |
|18     |Drama    |141      |
|38     |Comedy   |43       |
|70     |Action   |48       |
|148    |Drama    |25       |
|157    |Drama    |21       |
|161    |Drama    |31       |
|186    |Thriller |51       |
+-------+---------+---------+
only showing top 10 rows



#### Output Result Analysis

- Most active users tend to have a clear favourite genre, as seen by the "fav_genre" column in the last table.
- For example, user 148 rated 65 movies in total, and their most rated genre is "Drama" with 25 ratings. Similarly, users 463, 496, and 833 also have "Drama" as their favourite genre.
- The most popular genre among the top active users is "Drama", but there are also cases where "Action" is the favourite.
- The "max_count" column quantifies just how focused a user is on their favourite genre. For example, user 85 has rated 288 movies in total, and 152 of these were dramas, indicating a strong personal preference.
- This information provides insight into user preferences, and can be useful for personalized recommendations, marketing strategies, or understanding trends within the user community.

## iv) Find all the users who are less than 20 years old

In this part of the analysis, I focus on identifying the youngest users in the MovieLens 100k dataset.  
Age is an important demographic factor that can influence movie preferences and behavior on the platform.  
By selecting all users who are less than 20 years old, I can better understand the composition and characteristics of the teenage user group.

This information provides a basis for further exploration, such as comparing the preferences of younger users to those of older ones, or designing age-specific recommendation systems.  
Below, I use SQL queries in Spark to extract and summarize the relevant data for this age group.

### Use SQL to query all users under 20 years old

In [ ]:
users_under_20_df = spark.sql("""
    SELECT *
    FROM users
    WHERE age < 20
""")
users_under_20_df.show(10, truncate=False)

+-------+---+------+-------------+--------+
|user_id|age|gender|occupation   |zip_code|
+-------+---+------+-------------+--------+
|30     |7  |M     |student      |55436   |
|36     |19 |F     |student      |93117   |
|52     |18 |F     |student      |55105   |
|57     |16 |M     |none         |84010   |
|67     |17 |M     |student      |60402   |
|68     |19 |M     |student      |22904   |
|101    |15 |M     |student      |05146   |
|110    |19 |M     |student      |77840   |
|142    |13 |M     |other        |48118   |
|179    |15 |M     |entertainment|20755   |
+-------+---+------+-------------+--------+
only showing top 10 rows



### Statistics on gender and occupation distribution of users under 20 years old

In [ ]:
spark.sql("""
    SELECT gender, occupation, COUNT(*) AS cnt
    FROM users
    WHERE age < 20
    GROUP BY gender, occupation
    ORDER BY cnt DESC
""").show(truncate=False)

+------+-------------+---+
|gender|occupation   |cnt|
+------+-------------+---+
|M     |student      |37 |
|F     |student      |27 |
|M     |other        |3  |
|M     |none         |3  |
|M     |entertainment|2  |
|F     |writer       |2  |
|F     |artist       |1  |
|F     |salesman     |1  |
|F     |other        |1  |
+------+-------------+---+



#### Output Result Analysis

In this step, I selected all users whose age is less than 20 years old from the user dataset.  
The resulting table lists basic demographic information for each user, including user ID, age, gender, occupation, and zip code.

- The data show that most users under 20 are students, which matches expectations for this age group.
- There is a mix of male and female users, and both are well represented among the student population.
- A few users report occupations other than student, such as "entertainment," "writer," "artist," or "salesman." There are also cases where the occupation is missing ("none"), which might indicate incomplete user profiles.
- The age distribution in the sample covers teenagers (as young as 13) up to young adults just under 20.
- The results provide insight into the makeup of the youngest segment of MovieLens users, who may have different movie preferences and viewing habits compared to older users.

The aggregated statistics further show that the most common group is male students (37 users), followed by female students (27 users).  
There are only a few users in other occupational categories. This highlights that the majority of very young users in this dataset are still in school.

## v) Find all the users whose occupation is “scientist” and whose age is between 30 and 40 years old.

In this step, I focus on filtering the user data to find a specific subgroup.  
My goal is to select all users whose occupation is listed as "scientist" and whose age falls between 30 and 40 years old.  
To accomplish this, I use an SQL query on the `users` table.  
This approach enables me to precisely identify users who meet both conditions, which is valuable for targeted demographic analysis or downstream recommendation tasks.

After running the SQL query, I review the filtered results and count how many users match these criteria.  

### Use SQL to filter

In [ ]:
scientists_30_40_df = spark.sql("""
    SELECT *
    FROM users
    WHERE occupation = 'scientist' AND age BETWEEN 30 AND 40
""")
scientists_30_40_df.show(10, truncate=False)

+-------+---+------+----------+--------+
|user_id|age|gender|occupation|zip_code|
+-------+---+------+----------+--------+
|40     |38 |M     |scientist |27514   |
|71     |39 |M     |scientist |98034   |
|74     |39 |M     |scientist |T8H1N   |
|107    |39 |M     |scientist |60466   |
|183    |33 |M     |scientist |27708   |
|272    |33 |M     |scientist |53706   |
|309    |40 |M     |scientist |70802   |
|337    |37 |M     |scientist |10522   |
|430    |38 |M     |scientist |98199   |
|538    |31 |M     |scientist |21010   |
+-------+---+------+----------+--------+
only showing top 10 rows



### Count

In [ ]:
count = scientists_30_40_df.count()
print("Number of scientists aged 30 to 40: " + str(count))

Number of scientists aged 30 to 40: 16


The results show that there are **16 users** who satisfy both conditions.  
These users have user IDs such as 40, 74, 71, 107, 183, and others.  
All of them are male, as indicated by the gender column.  
Their zip codes are distributed across different regions, suggesting a certain level of geographic diversity even within this small professional and age-specific group.

## Insights and Explanations

This section provides a detailed summary of the main findings and analytical reasoning throughout the MovieLens 100k dataset exploration. Each analysis step yields important insights about user behavior and movie characteristics.

---

### 1. Calculating the Average Rating for Each Movie

By aggregating ratings across all users, I calculated the average rating and the total number of ratings for each movie.  
This process gives a clear picture of which movies are consistently favored or disfavored by the user base.  
Movies with a high number of ratings and a stable average rating are likely to have broad appeal or cultural significance.  
Conversely, movies with fewer ratings may represent niche interests or less popular genres.

---

### 2. Identifying the Top Ten Movies with the Highest Average Ratings

Focusing on movies with at least 50 ratings, I identified the top ten movies with the highest average scores.  
This threshold ensures the rankings are not biased by a small number of extreme ratings.  
Most top-ranked movies are either highly acclaimed classics or widely recognized by audiences.  
This finding demonstrates that collective user ratings can reliably highlight high-quality or iconic films.

---

### 3. Active Users and Their Favourite Genres

I filtered users who have rated at least 50 movies, labeling them as active or highly engaged users.  
For each of these users, I counted the number of ratings they gave in each genre, and then determined their most frequently rated (favourite) genre.

Insights include:
- Many active users consistently favor a specific genre, such as "Drama" or "Action".
- Some users have diverse interests, but still show a clear genre preference.
- The distribution of favorite genres among active users reflects overall audience tastes and possible trends in movie popularity.

---

### 4. Users Less Than 20 Years Old

By selecting users under the age of 20, I explored the demographic composition and behaviors of the youngest segment of the dataset.  
Most of these young users are students, with a relatively balanced distribution of genders.  
Understanding the preferences and activities of this age group can help with targeted recommendations or youth-focused analyses.

---

### 5. Users Who Are Scientists Aged 30-40

I applied a combined filter on both occupation and age to identify all users listed as "scientist" and aged between 30 and 40.  
This targeted filtering uncovers a specific professional demographic within the user base.  
Although only a small number of users meet these criteria, such focused subgroup analyses can be valuable for custom marketing strategies or understanding niche user groups.

---

### General Observations

- The MovieLens dataset contains a diverse set of users with varied demographic backgrounds and movie preferences.
- SQL and Spark operations enable powerful filtering and aggregation, making it possible to answer both broad and highly specific analytical questions.
- By combining multiple filters (such as age, occupation, and activity level), I can perform in-depth segmentation to support recommendations, marketing, or sociological research.
- Overall, data-driven exploration of movie ratings and user information reveals patterns in collective opinion, individual tastes, and demographic behaviors.



## Recommendations

Based on the analysis of the MovieLens 100k dataset and the key insights obtained from each step, I propose the following recommendations for future work, platform design, and research directions:

---

### 1. Improve Recommendation Systems with Popularity and Stability

Since movies with a high number of ratings and a stable average score generally reflect broad audience appeal, I recommend giving these movies more weight in collaborative filtering and popularity-based recommendation algorithms.  
Additionally, highlighting consistently popular movies can increase user satisfaction, especially for new or undecided users.

---

### 2. Adjust Genre Recommendations According to User Activity

The analysis shows that highly active users tend to have clear genre preferences.  
I recommend personalizing genre-based recommendations for these users, prioritizing their most frequently rated genres.  
For less active users, a diverse mix of genres may help them discover their specific interests.

---

### 3. Create Demographic-Specific Features and Marketing

Given the significant presence of young users (especially students under 20), I suggest developing youth-oriented features or movie collections (e.g., "Top Picks for Students" or "Movies for Young Audiences").  
Similarly, targeting users based on occupation (such as scientists or other professionals) can support niche content curation and specialized marketing campaigns.

---

### 4. Encourage Engagement among Inactive or New Users

Most high-engagement users contribute significantly to rating diversity and overall platform activity.  
To cultivate more active users, I recommend providing incentives for rating more movies, such as gamified achievements, badges, or personalized rewards for users who reach certain rating milestones.

---

### 5. Ensure Fairness in Ratings and Rankings

To avoid biases caused by a small number of ratings, always apply a minimum rating count threshold when displaying average scores or generating "top movie" lists.  
This approach ensures fairer and more reliable recommendations.

---

### 6. Enable Deeper User Segmentation for Advanced Analytics

The ability to filter by age, occupation, and other user attributes is powerful for research and platform optimization.  
I recommend maintaining and possibly expanding user profile fields, enabling even finer segmentation and targeted feature development in the future.

---

### 7. Continue Data Quality Checks and Enrichment

The analysis relies on the completeness and accuracy of user and movie metadata.  
I recommend routine data quality audits and, if possible, enriching the dataset with additional information such as more detailed genre tags, user watch history, or explicit feedback on recommendations.

---

**Summary:**  
Implementing these recommendations can improve the user experience, increase engagement, and allow for more precise movie suggestions and platform growth. Continuous data analysis should be a standard practice to adapt to evolving user interests and behaviors.


## Conclusion

In this project, I carried out a comprehensive analysis of the MovieLens 100k dataset using Spark SQL and PySpark DataFrame operations. My workflow covered every key stage, including data ingestion, schema definition, RDD and DataFrame transformations, and the application of advanced query logic. Throughout the process, I examined the characteristics of both movies and users in detail, deriving valuable insights for platform optimization and recommendation strategies.

---

### Key Findings

- **Movie Ratings:**  
  I successfully calculated the average rating for each movie, revealing both overall audience sentiment and the reliability of ratings based on rating count.

- **Top Rated Movies:**  
  By identifying the ten movies with the highest average ratings (with sufficient rating counts), I showed which titles are most appreciated by the community. This method also demonstrated the importance of accounting for popularity alongside rating averages.

- **User Activity & Genre Preference:**  
  I found that highly active users (those who have rated at least 50 movies) tend to show clear genre preferences. Most frequently, genres like "Drama" and "Action" stood out among top contributors, highlighting opportunities for more personalized recommendations.

- **User Demographics:**  
  My analysis of age and occupation revealed that a significant portion of the platform’s users are young, with students under 20 making up a considerable group. I also filtered out niche groups, such as scientists aged 30–40, to demonstrate how user segmentation can inform targeted marketing and content curation.

---

### Analytical Strengths

- The pipeline utilized PySpark’s distributed computing capabilities, enabling efficient processing of large datasets.
- I made use of both SQL and DataFrame APIs, allowing for flexible, powerful data exploration and transformation.
- Each major step included data validation and summary statistics, ensuring reliable interpretations and actionable insights.

---

### Final Thoughts

This end-to-end workflow demonstrates how big data technologies can unlock actionable knowledge from real-world user behavior and preferences.  
By systematically analyzing both movies and user groups, I have established a robust framework for data-driven improvements to recommendation platforms.  
Continuous analysis and iteration will be essential as audience interests evolve and as more data becomes available.